In [181]:
#%% Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bokeh # Bokeh is an interactive visualization library for modern web browsers. 
import folium # leaflet map visualization 

## What does thi code do:
## get the data
## visualize chart with matplotlib
## visualize leaflet map with folium
## create interactive map with bokeh on web browser

## Task 1: getting datasets

In [182]:
def get_data(data_name):
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{data_name}_global.csv'\
            .format(data_name = data_name)
    
    # Get data to dataframe
    data = pd.read_csv(url)
    
    #reshape data from wide to long data with melt function in pandas
    data = pd.melt(data,id_vars=data.columns[:4], var_name='Day', value_name=data_name)
    data[data_name].astype('int') # convert data to numeric
    return data

In [183]:
#%% Get data
confirmed_df = get_data('confirmed') # time_series_covid19_confirmed_global.csv

deaths_df = get_data('deaths') # time_series_covid19_deaths_global.csv

recovered_df = get_data('recovered') # time_series_covid19_recovered_global.csv

In [184]:
confirmed_df.head()
deaths_df.head()
recovered_df.head()

,Province/State,Country/Region,Lat,Long,Day,recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0


In [185]:
## merge Confirmed, death, and recovered into one dataframe
keys = ['Province/State','Country/Region', 'Lat','Long','Day']

all_data = confirmed_df
all_data = all_data.merge(deaths_df, on = keys, how = 'inner' )
all_data = all_data.merge(recovered_df, on = keys, how = 'inner' )
all_data['Day'] = pd.to_datetime(all_data['Day'])
all_data['Province/State'] = all_data['Province/State'].fillna('')

all_data['Province/State_Country/Region'] = all_data['Province/State'].fillna('')
all_data['Province/State_Country/Region'] = all_data['Province/State_Country/Region'].apply(lambda x: x if x == '' else x + ' - ')
all_data['Province/State_Country/Region'] = all_data['Province/State_Country/Region'] + all_data['Country/Region']
# delete unused variables.
#del confirmed_df, deaths_df, recovered_df, key



In [186]:
all_data['Province/State'].unique()
all_data['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India', 'Indone

In [187]:
# extract newest data
newest_data =  all_data[all_data['Day']== all_data.Day.max()]

In [188]:
newest_data

,Province/State,Country/Region,Lat,Long,Day,confirmed,deaths,recovered,Province/State_Country/Region
18620,,Afghanistan,33.000000,65.000000,2020-04-07,423,14,18,Afghanistan
18621,,Albania,41.153300,20.168300,2020-04-07,383,22,131,Albania
18622,,Algeria,28.033900,1.659600,2020-04-07,1468,193,113,Algeria
18623,,Andorra,42.506300,1.521800,2020-04-07,545,22,39,Andorra
18624,,Angola,-11.202700,17.873900,2020-04-07,17,2,2,Angola
18625,,Antigua and Barbuda,17.060800,-61.796400,2020-04-07,19,1,0,Antigua and Barbuda
18626,,Argentina,-38.416100,-63.616700,2020-04-07,1628,56,338,Argentina
18627,,Armenia,40.069100,45.038200,2020-04-07,853,8,87,Armenia
18628,Australian Capital Territory,Australia,-35.473500,149.012400,2020-04-07,96,2,40,Australian Capital Territory - Australia
18629,New South Wales,Australia,-33.868800,151.209300,2020-04-07,2686,21,4,New South Wales - Australia


## Task 2: Creat World map

In [189]:
lat, lon = 16.0,  108
zoom = 1.5
my_map = folium.Map(location=[lat, lon], zoom_start= zoom,
                 control_scale=True,
                 tiles= 'cartodbdark_matter')

In [190]:
# add mouse position
update_day = all_data.Day.max().strftime('%B %d %Y')
prefix = 'Data source: JHU. Update day: {update_day}.<br>Coordinates:'.format(update_day= update_day)
from folium.plugins import MousePosition
formatter = "function(num) {return L.Util.formatNum(num, 3) + '&#176';};"
MousePosition(
    position='topright',
    separator=' | ',
    empty_string='',
    lng_first=True,
    num_digits=20,
    prefix= prefix,
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(my_map)

## Task 3: Building the line chart over the time for each country

In [191]:
from bokeh.plotting import figure, show, output_file
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.formatters import NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.models import Legend
from bokeh.embed import file_html
from bokeh.resources import INLINE

# create mini chart with bokeh
# these chart will show as popup in the map
# 
def popup_plot(plot_df):
    
    #plot_df = all_data[all_data['Country/Region'] == 'US']
    plot_df['Daystr'] = plot_df['Day'].dt.strftime('%m/%d')

    tooltip = '''<div style="color: #736E6D; text-align:left;">Confirmed: <span style="color:red"><b> @top</b> </span></div >'''

    tools = "crosshair,pan,reset, save,wheel_zoom"
    hover = HoverTool(tooltips=tooltip, mode='vline')
    #output_file('vbar.html')

    # declare a chart
    p = figure(plot_width=600, plot_height=200,
              tools = [hover, tools],
              x_axis_type= "datetime")

    # set value to the chart
    p.vbar(x=plot_df.Day, width = 3, bottom=0,
           top=plot_df.confirmed, color="firebrick")

    # set annotations: title, x,y lable, legend
    p.xaxis.formatter=DatetimeTickFormatter(days=["%m/%d"])
    p.yaxis.formatter = NumeralTickFormatter(format="0")
    html = file_html(p, INLINE, "my plot")
    bokeh.io.curdoc().clear()  
    bokeh.io.state.State().reset()
    bokeh.io.reset_output()
    
    return html


In [206]:
plot_df = all_data[all_data['Country/Region'] == 'US']
plot_df['Daystr'] = plot_df['Day'].dt.strftime('%m/%d')

tooltips=[
        ( 'Confirmed' , '@confirmed'),
        ( 'Recovered', '@recovered'),
        ( 'Deaths', '@death')
        ]

tools = "crosshair,pan,reset, save,wheel_zoom"
hover = HoverTool(tooltips=tooltips, mode='vline')
#output_file('vbar.html')

# declare a chart
p = figure(plot_width=600, plot_height=200,
          tools = [hover, tools],
          x_axis_type= "datetime")

# add some renderers
confirmed = plot_df.confirmed
recovered = plot_df.recovered
deaths = plot_df.deaths


p.line(x=plot_df.Day, y=confirmed ,legend_label="Confirmed", line_color="red", line_width=3)
p.line(x=plot_df.Day, y =recovered, legend_label="Recovered", line_color="green")
p.line(x=plot_df.Day, y= deaths, legend_label="Death", line_color="orange", line_width=3)

p.legend.location = "top_left"
# set annotations: title, x,y lable, legend
p.xaxis.formatter=DatetimeTickFormatter(days=["%m/%d"])
p.yaxis.formatter = NumeralTickFormatter(format="0")

bokeh.io.curdoc().clear()  
bokeh.io.state.State().reset()
bokeh.io.reset_output()

show(p)

C:\Users\quoch\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [192]:
## add data to folium map
from folium import IFrame
for i in range(newest_data.shape[0]/2):
    
    # gather information
    lat = newest_data.Lat.iloc[i]
    long = newest_data.Long.iloc[i]
    
    if newest_data.confirmed.iloc[i] > 1000:
        radius = int(3 * newest_data.confirmed.iloc[i]) # radius of the circle
    else:
        radius = int(3  * newest_data.confirmed.iloc[i] * 20)
    
    
    if newest_data['Province/State'].iloc[i]=='':
        state = newest_data['Country/Region'].iloc[i]
    else:
        state = newest_data['Province/State_Country/Region'].iloc[i]
        
    confirmed = newest_data['confirmed'].iloc[i]
    recovered = newest_data['recovered'].iloc[i]
    deaths = newest_data['deaths'].iloc[i]
    existing = confirmed - recovered - deaths
    
    # Format toolstip
    tooltip = '''<div style="background-color: black">
                <p style="text-align: center; color: #FFFFFF "><strong>{state}</strong></p>
                <div style="color: #736E6D; text-align:left;">Confirmed: <span style="color:red"><b> {confirmed}</b> </span></div >
                <div style="color: #736E6D; text-align:left;">Deaths: <span ><b> {deaths}</b> </span></div >
                <div style="color: #736E6D; text-align:left;">Recovered: <span style="color:#37FA02"><b> {recovered}</b> </span></div>
                <div style="color: #736E6D; text-align:left;">Existing: <span style="color:#FA7202"><b>{existing}</b> </span></div>
                </div>
            '''
    # assign value to the toolstip
    tooltip = tooltip.format(state= state.upper(), confirmed= confirmed,
                             recovered= recovered, deaths= deaths,
                             existing= existing)
    
    # Popup chart
# my computer with 8GB of Ram can not run the map with popup chart    
    plot_df = all_data.loc[all_data['Country/Region'] == newest_data['Country/Region'].iloc[i]] 
    popuphtml = popup_plot(plot_df)
    iframe = IFrame(html=popuphtml, width=500, height=200)
    popup = folium.Popup(iframe, max_width=600, max_height= 250)
    
    ## Generate the circle
    folium.Circle(
            location= [lat,long],
            popup=popup, # I will generate the chart of the country as a popup
            radius=radius,
            color= '#F7F7F7',
            weight= 0.2,
            fill= True,
            fill_color= '#ff0000',
            fillOpacity = 1,
            tooltip =tooltip
           ).add_to(my_map)

    

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
my_map